In [1]:
import cv2
import os
import uuid
import time
import numpy as np

In [3]:
IMAGE_PATH = "Tensorflow/workspace/images/collectedimages/"
LABEL_PATH = "Tensorflow/workspace/annotations/"

In [4]:
labels = ["hello", "yes", "no", "thanks", "iloveyou", "mamamia"]
num_imgs =  15

# Collecting and Labeling Images

In [ ]:
for label in labels:
    !mkdir {"Tensorflow\workspace\images\collectedimages\\" + label}
    cap = cv2.VideoCapture(0)QQQQ
    print(f"Collecting Images for {label}")
    time.sleep(5)
    for img_num in range(num_imgs):
        ret, frame = cap.read()
        image_name = os.path.join(IMAGE_PATH, label, f"{label}.{str(uuid.uuid1())}.jpg")
        cv2.imwrite(image_name, frame)
        cv2.imshow("frame", frame)
        time.sleep(2)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            cv2.destroyAllWindows()
    
    cap.release()        

In [2]:
!labelimg

Cancel creation.
Cancel creation.
Cancel creation.
Image:C:\Users\Batlagh\Projects\object-detection\Tensorflow\workspace\images\train\hello.3ab23416-1a79-11ef-81cd-f02f74f4bb66.jpg -> Annotation:C:/Users/Batlagh/Projects/object-detection/Tensorflow/workspace/images/train\hello.3ab23416-1a79-11ef-81cd-f02f74f4bb66.xml
Image:C:\Users\Batlagh\Projects\object-detection\Tensorflow\workspace\images\train\hello.3be77589-1a79-11ef-a30d-f02f74f4bb66.jpg -> Annotation:C:/Users/Batlagh/Projects/object-detection/Tensorflow/workspace/images/train\hello.3be77589-1a79-11ef-a30d-f02f74f4bb66.xml
Image:C:\Users\Batlagh\Projects\object-detection\Tensorflow\workspace\images\train\hello.3d1dfbf4-1a79-11ef-b70f-f02f74f4bb66.jpg -> Annotation:C:/Users/Batlagh/Projects/object-detection/Tensorflow/workspace/images/train\hello.3d1dfbf4-1a79-11ef-b70f-f02f74f4bb66.xml
Image:C:\Users\Batlagh\Projects\object-detection\Tensorflow\workspace\images\train\hello.3e56fd9c-1a79-11ef-b576-f02f74f4bb66.jpg -> Annotation:C

# Split Images and Labels to Train and Test

In [6]:
WORKSPACE_PATH = 'Tensorflow/workspace'
SCRIPTS_PATH = 'Tensorflow/scripts'
APIMODEL_PATH = 'Tensorflow/models'
ANNOTATION_PATH = WORKSPACE_PATH+'/annotations'
IMAGE_PATH = WORKSPACE_PATH+'/images'
MODEL_PATH = WORKSPACE_PATH+'/models'
PRETRAINED_MODEL_PATH = WORKSPACE_PATH+'/pre-trained-models'
CONFIG_PATH = MODEL_PATH+'/my_ssd_mobnet/pipeline.config'
CHECKPOINT_PATH = MODEL_PATH+'/my_ssd_mobnet/'
CUSTOM_MODEL_NAME = "my_ssd_mobnet"

# Create Label Map

In [7]:
label_map = [{"name": "hello", "id": 1},
             {"name": "yes", "id": 2},
             {"name": "no", "id": 3},
             {"name": "thanks", "id": 4},
             {"name": "i love you", "id": 5},
             {"name": "mamamia", "id": 6}]

with open(ANNOTATION_PATH + '\label_map.pbtxt', 'w') as f:
    for label in label_map:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

# Create TF Records

In [9]:
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x {IMAGE_PATH + '/train'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/train.record'}
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x{IMAGE_PATH + '/test'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/test.record'}

Successfully created the TFRecord file: Tensorflow/workspace/annotations/train.record
Successfully created the TFRecord file: Tensorflow/workspace/annotations/test.record


# Update Config For Transfer Learning

In [7]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [8]:
CONFIG_PATH = MODEL_PATH+'/'+CUSTOM_MODEL_NAME+'/pipeline.config'

In [12]:
config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
config

{'model': ssd {
   num_classes: 6
   image_resizer {
     fixed_shape_resizer {
       height: 320
       width: 320
     }
   }
   feature_extractor {
     type: "ssd_mobilenet_v2_fpn_keras"
     depth_multiplier: 1.0
     min_depth: 16
     conv_hyperparams {
       regularizer {
         l2_regularizer {
           weight: 3.9999998989515007e-05
         }
       }
       initializer {
         random_normal_initializer {
           mean: 0.0
           stddev: 0.009999999776482582
         }
       }
       activation: RELU_6
       batch_norm {
         decay: 0.996999979019165
         scale: true
         epsilon: 0.0010000000474974513
       }
     }
     use_depthwise: true
     override_base_feature_extractor_hyperparams: true
     fpn {
       min_level: 3
       max_level: 7
       additional_layer_depth: 128
     }
   }
   box_coder {
     faster_rcnn_box_coder {
       y_scale: 10.0
       x_scale: 10.0
       height_scale: 5.0
       width_scale: 5.0
     }
   }
   match

In [13]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(CONFIG_PATH, "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

In [14]:
pipeline_config.model.ssd.num_classes = 6
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0'
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path = ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/train.record']
pipeline_config.eval_input_reader[0].label_map_path = ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/test.record']

In [15]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(CONFIG_PATH, "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

# Train the Model

In [16]:
print("""python {}/research/object_detection/model_main_tf2.py --model_dir={}/{} --pipeline_config_path={}/{}/pipeline.config --num_train_steps=10000""".format(APIMODEL_PATH, MODEL_PATH,CUSTOM_MODEL_NAME,MODEL_PATH,CUSTOM_MODEL_NAME))

python Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workspace/models/my_ssd_mobnet --pipeline_config_path=Tensorflow/workspace/models/my_ssd_mobnet/pipeline.config --num_train_steps=10000


# Load Trained Model From the Checkpoints

In [9]:
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(CHECKPOINT_PATH, 'ckpt-12')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

# Detect in Real-Time

In [11]:
category_index = label_map_util.create_category_index_from_labelmap(ANNOTATION_PATH + "/label_map.pbtxt")

In [17]:
cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

In [18]:
while True: 
    ret, frame = cap.read()
    image_np = np.array(frame)
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                min_score_thresh=.5,
                agnostic_mode=False)

    cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break